In [73]:
import openrouteservice as ors
import pandas as pd
import folium
import geopandas as gpd

import copy

In [74]:
import constants
import fleet_manager as fm
import utils


In [127]:
pd.options.display.max_rows = 1000    # Show up to 1000 rows
pd.options.display.max_columns = 50   # Show up to 50 columns
pd.set_option('display.max_colwidth', None) # Show all contents in the cells

In [132]:
# change the decimal precision --> default = 6 digits
gpd.options.display_precision = 16 

In [75]:
# create 2-3 postcode options, and prepare pothole datasets accordingly.
"LS9 0BT", "LS6 1PP", "LS6 3HE"
postcode = 'LS9 0BT' 


# only options: 3, 5, 10 (drop-down menu)
bot_count = 3

customise = False

In [76]:
client = ors.Client(constants.API_KEY)
client

<table border=1>
<tr>
<td><b>Library	<td><b>Coordinate Order	
<tr>
<td>Folium	<td>Latitude, Longitude	
<tr>
<td>ORS	<td>Longitude, Latitude	

In [77]:
# find the postcode coordinates
station = client.pelias_search(text=postcode)
station

{'geocoding': {'version': '0.2',
  'attribution': 'https://openrouteservice.org/terms-of-service/#attribution-geocode',
  'query': {'text': 'LS9 0BT',
   'size': 10,
   'private': False,
   'lang': {'name': 'English',
    'iso6391': 'en',
    'iso6393': 'eng',
    'via': 'default',
    'defaulted': True},
   'querySize': 20,
   'parser': 'libpostal',
   'parsed_text': {'postalcode': 'ls9 0bt'}},
  'engine': {'name': 'Pelias', 'author': 'Mapzen', 'version': '1.0'},
  'timestamp': 1712874748288},
 'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-1.517644, 53.789386]},
   'properties': {'id': '470842435',
    'gid': 'whosonfirst:postalcode:470842435',
    'layer': 'postalcode',
    'source': 'whosonfirst',
    'source_id': '470842435',
    'name': 'LS9 0BT',
    'postalcode': 'LS9 0BT',
    'postalcode_gid': 'whosonfirst:postalcode:470842435',
    'confidence': 1,
    'match_type': 'exact',
    'accuracy': 'centroid',
    'co

In [78]:
station_coords = station['features'][0]['geometry']['coordinates']
station_coords

[-1.517644, 53.789386]

In [80]:
m = folium.Map(location=list(reversed(station_coords)), tiles="cartodbpositron", zoom_start=16)

# Create icon using folium.Icon for station from where all bots start and end
icon=BeautifyIcon(
        icon_shape='marker',
        text_color='black',
        background_color="#FFF",
        inner_icon_style="font-size:12px;padding-top:-5px;",
        icon='flag-checkered'
    )

# Add marker with the created icon to the map
folium.Marker(location=list(reversed(station_coords)), icon=icon).add_to(m)

# Display the map
m

In [81]:
fleet = fm.FleetManager(bot_count=bot_count, start=list(reversed(station_coords)))
fleet

In [145]:
potholes_details = gpd.read_file('LS90BT_potholes_small.json')
potholes_details

defect          dim                                         geometry
0  pothole    1x0.5x0.2  POINT (-1.5202653014356144 53.7886094292194912)
1  pothole  0.5x1.2x0.2  POINT (-1.5185276122489597 53.7884573448203867)
2  pothole  0.5x1.2x0.2  POINT (-1.5181200061435902 53.7886537870654990)
3  pothole    1x0.5x0.2  POINT (-1.5165367499993749 53.7894171973833295)
4  pothole    1x0.5x0.2  POINT (-1.5158937220954840 53.7902010524229013)
5  pothole    2x0.5x0.2  POINT (-1.5157207769570675 53.7901240147059525)
6  pothole    1x0.5x0.2  POINT (-1.5150777490531766 53.7897200289646946)
7  pothole    1x0.5x0.2  POINT (-1.5165475271180355 53.7900878370366797)
8  pothole    1x0.5x0.1  POINT (-1.5178539965092739 53.7884356902426077)

In [111]:
potholes_details[['defect']].value_counts()

defect 
pothole    9
dtype: int64

In [110]:
potholes_details[['dim']].value_counts()

dim        
1x0.5x0.2      5
0.5x1.2x0.2    2
1x0.5x0.1      1
2x0.5x0.2      1
dtype: int64

In [128]:
potholes_data = pd.read_json('LS90BT_potholes_small.json')
potholes_data

type  \
0  FeatureCollection   
1  FeatureCollection   
2  FeatureCollection   
3  FeatureCollection   
4  FeatureCollection   
5  FeatureCollection   
6  FeatureCollection   
7  FeatureCollection   
8  FeatureCollection   

                                                                                                                                                                         features  
0    {'type': 'Feature', 'properties': {'defect': 'pothole', 'dim': '1x0.5x0.2'}, 'geometry': {'coordinates': [-1.520265301435614, 53.78860942921949], 'type': 'Point'}, 'id': 0}  
1  {'type': 'Feature', 'properties': {'defect': 'pothole', 'dim': '0.5x1.2x0.2'}, 'geometry': {'coordinates': [-1.518527612248959, 53.78845734482039], 'type': 'Point'}, 'id': 1}  
2    {'type': 'Feature', 'properties': {'defect': 'pothole', 'dim': '0.5x1.2x0.2'}, 'geometry': {'coordinates': [-1.51812000614359, 53.7886537870655], 'type': 'Point'}, 'id': 2}  
3    {'type': 'Feature', 'properties': {'defect': 'pothole', 'dim': '1x0.5x0.2'}, 'geometry': {'coordinates': [-1.516536749999375, 53.78941719738333], 'type': 'Point'}, 'id': 3}  
4     {'type': 'Feature', 'properties': {'defect': 'pothole', 'dim': '1x0.5x0.2'}, 'geometry': {'coordinates': [-1.515893722095484, 53.7902010524229], 'type': 'Point'}, 'id': 4}  
5    {'type': 'Feature', 'properties': {'defect': 'pothole', 'dim': '2x0.5x0.2'}, 'geometry': {'coordinates': [-1.515720776957067, 53.79012401470595], 'type': 'Point'}, 'id': 5}  
6   {'type': 'Feature', 'properties': {'defect': 'pothole', 'dim': '1x0.5x0.2'}, 'geometry': {'coordinates': [-1.515077749053176, 53.789720028964695], 'type': 'Point'}, 'id': 6}  
7    {'type': 'Feature', 'properties': {'defect': 'pothole', 'dim': '1x0.5x0.2'}, 'geometry': {'coordinates': [-1.516547527118035, 53.79008783703668], 'type': 'Point'}, 'id': 7}  
8    {'type': 'Feature', 'properties': {'defect': 'pothole', 'dim': '1x0.5x0.1'}, 'geometry': {'coordinates': [-1.517853996509273, 53.78843569024261], 'type': 'Point'}, 'id': 8}

In [84]:
ut = utils.Utils()
ut

In [85]:
# calculate estimates
pothole_estimates = [] # vol, surface_area, qty_reqd, setup_time, service_time

In [86]:
# create jobs
jobs = []

for feature in potholes_data['features']:
    volume, surface_area, qty_reqd, setup_time, service_time = ut.calculate_eta(feature)
    pothole_estimates.append({
        'id': feature['id'], 
        'defect': feature['properties']['defect'], 
        'volume': volume, 
        'surface_area': surface_area, 
        'qty_reqd': qty_reqd, 
        'setup_time': setup_time, 
        'service_time': service_time})
    jobs.append(ors.optimization.Job(
        location=feature['geometry']['coordinates'], 
        id=feature['id'], 
        setup=setup_time, 
        service=service_time, 
        amount=[qty_reqd])
        )

In [87]:
pothole_estimates

[{'id': 0,
  'defect': 'pothole',
  'volume': 0.1,
  'surface_area': 0.5,
  'qty_reqd': 240.0,
  'setup_time': 600,
  'service_time': 90.0},
 {'id': 1,
  'defect': 'pothole',
  'volume': 0.12,
  'surface_area': 0.6,
  'qty_reqd': 288.0,
  'setup_time': 600,
  'service_time': 108.0},
 {'id': 2,
  'defect': 'pothole',
  'volume': 0.12,
  'surface_area': 0.6,
  'qty_reqd': 288.0,
  'setup_time': 600,
  'service_time': 108.0},
 {'id': 3,
  'defect': 'pothole',
  'volume': 0.1,
  'surface_area': 0.5,
  'qty_reqd': 240.0,
  'setup_time': 600,
  'service_time': 90.0},
 {'id': 4,
  'defect': 'pothole',
  'volume': 0.1,
  'surface_area': 0.5,
  'qty_reqd': 240.0,
  'setup_time': 600,
  'service_time': 90.0},
 {'id': 5,
  'defect': 'pothole',
  'volume': 0.2,
  'surface_area': 1.0,
  'qty_reqd': 480.0,
  'setup_time': 600,
  'service_time': 180.0},
 {'id': 6,
  'defect': 'pothole',
  'volume': 0.1,
  'surface_area': 0.5,
  'qty_reqd': 240.0,
  'setup_time': 600,
  'service_time': 90.0},
 {'id': 

### Potholes estimates

In [88]:
pd.DataFrame(pothole_estimates)

id   defect  volume  surface_area  qty_reqd  setup_time  service_time
0   0  pothole    0.10           0.5     240.0         600          90.0
1   1  pothole    0.12           0.6     288.0         600         108.0
2   2  pothole    0.12           0.6     288.0         600         108.0
3   3  pothole    0.10           0.5     240.0         600          90.0
4   4  pothole    0.10           0.5     240.0         600          90.0
5   5  pothole    0.20           1.0     480.0         600         180.0
6   6  pothole    0.10           0.5     240.0         600          90.0
7   7  pothole    0.10           0.5     240.0         600          90.0
8   8  pothole    0.05           0.5     120.0         600          75.0

In [89]:
jobs

In [90]:
# plan route using ORS
route_plan = ors.optimization.optimization(client,
                                  jobs=jobs,
                                  vehicles=fleet.bots,
                                  shipments=None,
                                  matrix=None,
                                  geometry=True,
                                  dry_run=None)

# if you get error code 500 from ORS API, check location coordinate format.

route_plan

{'code': 0,
 'summary': {'cost': 183,
  'routes': 3,
  'unassigned': 3,
  'delivery': [1368],
  'amount': [1368],
  'pickup': [0],
  'setup': 0,
  'service': 543,
  'duration': 183,
  'waiting_time': 0,
  'priority': 0,
  'distance': 944,
  'violations': [],
  'computing_times': {'loading': 84, 'solving': 55, 'routing': 39}},
 'unassigned': [{'id': 5,
   'location': [-1.515720776957067, 53.79012401470595],
   'type': 'job'},
  {'id': 2, 'location': [-1.51812000614359, 53.7886537870655], 'type': 'job'},
  {'id': 0,
   'location': [-1.520265301435614, 53.78860942921949],
   'type': 'job'}],
 'routes': [{'vehicle': 0,
   'cost': 36,
   'delivery': [408],
   'amount': [408],
   'pickup': [0],
   'setup': 0,
   'service': 183,
   'duration': 36,
   'waiting_time': 0,
   'priority': 0,
   'distance': 201,
   'steps': [{'type': 'start',
     'location': [-1.517644, 53.789386],
     'setup': 0,
     'service': 0,
     'waiting_time': 0,
     'load': [408],
     'arrival': 0,
     'duration': 0

In [91]:
# TODO
# check for how many jobs are left undone
# try with higher number of bots
# try with higher capacity
#  try with bigger time_window

In [92]:
# plot potholes(jobs) and routes
for pothole in potholes_data['features']:
    folium.Marker(
        location=list(reversed(pothole['geometry']['coordinates'])),
        tooltip=pothole['id'],
        icon=folium.Icon(color='red', icon='info-sign')
        # add details in tooltip
        # add routes
        # add layer control
    ).add_to(m)

m

In [147]:
line_colours = ['orange', 'blue', 'green', 'pink', 'yellow']

for route in route_plan['routes']:
    folium.PolyLine(
        locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']], 
        weight=5, 
        color=line_colours[route['vehicle']], 
        opacity=0.5
    ).add_to(m)

m

In [62]:
line_colours = ['orange', 'blue', 'green', 'pink', 'yellow']

for route in route_plan['routes']:
    r = ors.convert.decode_polyline(route['geometry'])['coordinates']
    folium.PolyLine(locations=r, weight=5, color='blue').add_to(m)

m

TypeError: string indices must be integers, not 'str'

In [ ]:
# maps, charts, comparisons


In [99]:
# read from the json response

import matplotlib.pyplot as plt

# Data from ORS response
routes = route_plan['routes']

# Lists to store data for charts
distances = []
durations = []
setup_times = []
service_times = []

# Extract data from each route
for route in routes:
  distances.append(route['distance'])
  durations.append(route['duration'])
  for step in route['steps']:
    setup_times.append(step['setup'])
    service_times.append(step['service'])

# Distance vs. Time Chart
plt.figure(figsize=(10, 6))
plt.scatter(distances, durations)
plt.xlabel('Distance (meters)')
plt.ylabel('Duration (seconds)')
plt.title('Distance vs. Time per Route')
plt.grid(True)
plt.savefig('distance_vs_time.png')
plt.close()

# Setup Time vs. Service Time (per bot)
bot_setup_times = {}
bot_service_times = {}

# Separate data by bot (vehicle)
for route in routes:
  vehicle = route['vehicle']
  for step in route['steps']:
    if vehicle not in bot_setup_times:
      bot_setup_times[vehicle] = []
      bot_service_times[vehicle] = []
    bot_setup_times[vehicle].append(step['setup'])
    bot_service_times[vehicle].append(step['service'])

# Create separate charts for each bot
for bot, setup_data in bot_setup_times.items():
  plt.figure(figsize=(8, 5))
  plt.plot(setup_data, label='Setup Time')
  plt.plot(bot_service_times[bot], label='Service Time')
  plt.xlabel('Job Index')
  plt.ylabel('Time (seconds)')
  plt.title(f'Setup vs. Service Time (Bot {bot})')
  plt.legend()
  plt.grid(True)
  plt.savefig(f'bot_{bot}_setup_vs_service.png')
  plt.close()

# Setup Time vs. Service Time (per pothole) - requires pothole IDs
# ... (implement logic to associate setup/service times with pothole IDs)


In [ ]:
# show charts here

In [100]:
# tableau dashboard? --> NO
# instead grid view with 4 or 6 charts.

In [ ]:
# collapse bot_id=1 at time t=2*3600 (2hr)

# re-scheduling for available bots (bot_ids=[0, 2]; [capacity=] [jobs_pending=] [current_location=])